<a href="https://colab.research.google.com/github/NandunPerera/flask-backend-detection/blob/main/Copy_of_Copy_of_flask_age_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Paths

In [ ]:
# dir_ = '/content/drive/MyDrive/flask/models/'
# gender = dir_ +  'model_gender.h5'
# age_v2 = dir_ +  'model_age_v2.h5'
# age_v1 = dir_ +  'age_mode_vN.h5'

YOUR_APP_NAME = "g10-age-gender"
url_gen = f'https://{YOUR_APP_NAME}.herokuapp.com/v1/models/gender:predict'
url_age = f'https://{YOUR_APP_NAME}.herokuapp.com/v1/models/age:predict'

# INDEX_PATH = '/content/drive/MyDrive/flask/'

# App Code

In [ ]:
import sys
import os
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import load_model

In [ ]:
from flask import Flask, redirect, url_for, request, render_template
from flask_ngrok import run_with_ngrok

In [ ]:
# Imports
import dlib
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

In [ ]:
import requests
import json

In [ ]:
def detect_faces(image):
    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()

    # Run detector and get bounding boxes of the faces on image.
    detected_faces = face_detector(image, 1)
    face_frames = [(x.left(), x.top(), x.right(), x.bottom()) for x in
                   detected_faces]

    return face_frames

In [ ]:
def make_prediction(instances, many=False,isAge = False):
    if not many:
        data = json.dumps({"signature_name": "serving_default", "instances": [instances.tolist()]})
    else:
        data = json.dumps({"signature_name": "serving_default", "instances": instances.tolist()})
    headers = {"content-type": "application/json"}
    if(not isAge):
      json_response = requests.post(url_gen, data=data, headers=headers)
    else:
      json_response = requests.post(url_age, data=data, headers=headers)
    jj = json.loads(json_response.text)
    return jj

# External Predict

In [ ]:
# def model_predict_serving(img_path):
#     test_image = io.imread(img_path)
#     detected_faces = detect_faces(test_image)

#     classes = [ 'Male' , 'Female' ]

#     for n, face_rect in enumerate(detected_faces):
#         face = Image.fromarray(test_image).crop(face_rect)
#         face_array = tf.keras.preprocessing.image.img_to_array(face)
#         face_img = tf.keras.preprocessing.image.array_to_img(face_array)

#         dim_age = (200, 200)
#         dim_gender = (128, 128)
#         resized_age_arr = tf.keras.preprocessing.image.smart_resize(face_array,size=dim_age) / 255
#         resized_gender_arr = tf.keras.preprocessing.image.smart_resize(face_array,size=dim_gender) / 255

#         p_gender =  make_prediction(np.expand_dims( resized_gender_arr , 0 ),many = True)
#         p_age = make_prediction(np.expand_dims( resized_age_arr , 0 ),many = True, isAge= True)

#         gender = classes[np.argmax(p_gender['predictions'][0])]
#         age = round((p_age['predictions'][0])[0] * 116)
#         print(age,gender)
        
#         face_img = tf.keras.preprocessing.image.array_to_img(resized_age_arr)

#         return f'Gender: {gender} | Age: {age}'

# Image preprocessing nandun

In [ ]:
# def loadImage(filepath):
#   test_img = image.load_img(filepath, target_size=(200, 200))
#   test_img = image.img_to_array(test_img)
#   test_img = np.expand_dims(test_img, axis = 0)
#   test_img /= 255
#   return test_img

# Local Predict

In [ ]:
!pip install fer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 812 kB 28.6 MB/s 
     |████████████████████████████████| 2.3 MB 58.0 MB/s 


In [ ]:
from fer import FER

In [ ]:
emotion_detector = FER(mtcnn=True)

In [ ]:
# pip install deepface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 87 kB 7.6 MB/s 
     |████████████████████████████████| 60.9 MB 1.2 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [ ]:
# from deepface import DeepFace

ModuleNotFoundError: ignored

In [ ]:
def predict_emotion(img_path):
    # face_analysis = DeepFace.analyze(img_path)

    test_img = io.imread(img_path)

    dominant_emotion, emotion_score = emotion_detector.top_emotion(test_img)
    
    return dominant_emotion


    # return f'emotion{face_analysis["dominant_emotion"]}| age:{face_analysis["age"]}'

In [ ]:
# def model_predict(img_path):
#     test_image = io.imread(img_path)
#     detected_faces = detect_faces(test_image)

#     classes = [ 'Male' , 'Female' ]

#     for n, face_rect in enumerate(detected_faces):
#         face = Image.fromarray(test_image).crop(face_rect)
#         face_array = tf.keras.preprocessing.image.img_to_array(face)
#         face_img = tf.keras.preprocessing.image.array_to_img(face_array)

#         dim_age = (200, 200)
#         dim_gender = (128, 128)
#         resized_age_arr = tf.keras.preprocessing.image.smart_resize(face_array,size=dim_age) / 255
#         resized_gender_arr = tf.keras.preprocessing.image.smart_resize(face_array,size=dim_gender) / 255

#         pred_gen = model_gender.predict( np.expand_dims( resized_gender_arr , 0 ) )[0]
#         pred_age_v1 = int(model_age_v1.predict(loadImage(img_path) )*116)
#         pred_age_v2 = model_age_v2.predict( np.expand_dims( resized_age_arr , 0 ) )

#         print(pred_age_v2[0] * 116 , pred_age_v2[1] * 116)
        
#         label_g = classes[ np.argmax( pred_gen ) ]
#        # label_a_v1 = round(pred_age_v1[0].tolist()[0] * 116)
#         label_a_v1 = pred_age_v1
#         label_a_v2 = round((((pred_age_v2[0] * 116 + pred_age_v2[1] * 116)/2)[0]).tolist()[0])
#         face_img = tf.keras.preprocessing.image.array_to_img(resized_age_arr)
#         label_emotion = predict_emotion(img_path)



#         return f'Gender: {label_g} | Age(V1): {label_a_v1} | Age(V2): {label_a_v2} | emotion: {label_emotion}'

# External with Alignments

In [ ]:
# predictor_path = '/content/drive/MyDrive/flask/shape_predictor_68_face_landmarks.dat'

In [ ]:
def model_predict_serving_align(img_path):
    classes = [ 'Male' , 'Female' ]

    results = []

    detector = dlib.get_frontal_face_detector()
    sp = dlib.shape_predictor(predictor_path)

    test_img = io.imread(img_path)
    dets = detector(test_img, 1)

    num_faces = len(dets)
    if num_faces == 0:
      return "Sorry, there were no faces found"
      
    
    #detected_faces = detect_faces(test_image)

    faces = dlib.full_object_detections()
    for detection in dets:
        faces.append(sp(test_img, detection))
    

    images = dlib.get_face_chips(test_img, faces,size = 200,padding = 0.25 )

    for face in images:
        plt.subplot(1, len(images), n+1)
        plt.axis('off')
        plt.imshow(face)

        face_array = tf.keras.preprocessing.image.img_to_array(face)
        face_img = tf.keras.preprocessing.image.array_to_img(face_array)

        dim_age = (200, 200)
        dim_gender = (128, 128)
        resized_age_arr = tf.keras.preprocessing.image.smart_resize(face_array,size=dim_age) / 255
        resized_gender_arr = tf.keras.preprocessing.image.smart_resize(face_array,size=dim_gender) / 255

        p_gender =  make_prediction(np.expand_dims( resized_gender_arr , 0 ),many = True)
        p_age = make_prediction(np.expand_dims( resized_age_arr , 0 ),many = True, isAge= True)

        gender = classes[np.argmax(p_gender['predictions'][0])]
        age = round((p_age['predictions'][0])[0] * 116)
        label_emotion = predict_emotion(img_path)
        print(age,gender)
        
        face_img = tf.keras.preprocessing.image.array_to_img(resized_age_arr)

        return f'Gender: {gender}|Age: {age}|emotion:{label_emotion}'

# Flask Code

In [ ]:
# Load your trained model
model_gender = keras.models.load_model(gender)
model_age_v1 = keras.models.load_model(age_v1)
model_age_v2 = keras.models.load_model(age_v2)

In [ ]:
#Template Folder Path
TEMPLATE_PATH = '/template'
STATIC_PATH = '/static'

In [ ]:
# Define a flask app
app = Flask(__name__,template_folder=TEMPLATE_PATH,static_folder=STATIC_PATH)
# run_with_ngrok(app)  # Start ngrok when app is run

In [ ]:
# !pip install pyngrok==4.1.1

In [ ]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

In [ ]:
# !tar -xvf /content/ngrok-stable-linux-amd64.tgz

In [ ]:
# !./ngrok authtoken 28mYPYUE6RZ4JilKi7A0wUqIlHU_7B3KXBbRG5xBBCGq8zdnj

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# @app.route('/', methods=['GET'])
# def index():
#     # Main page
#     return render_template('index.html')

@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']
        print(f.filename)
        # Save the file to ./uploads
        basepath = '/content'
        file_path = os.path.join(
            basepath, 'sample_data', f.filename)
        f.save(file_path)
        print(file_path)
        # Make prediction | Involves Model
        # predictions = model_predict(file_path) #Predict Locally
       predictions = model_predict_serving_align(file_path) #Predict with TFServing
        print(predictions)

        return predictions 

    return None


if __name__ == '__main__':
    app.run()